<a href="https://colab.research.google.com/github/eternityduck/ML_KPI/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from google.colab import files
import zipfile
from tensorflow.keras.optimizers import RMSprop

drive.mount('/content/drive')

local_zip = 'drive/MyDrive/KPI/AI/cats-or-dogs.zip'  #'./cats-or-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./cats-or-dogs')
zip_ref.close()



## Pipeline

In [ ]:
import os
import shutil

shutil.rmtree("./cats-or-dogs/.ipynb_checkpoints")
shutil.rmtree("./validation-cats-or-dogs/.ipynb_checkpoints")

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        './cats-or-dogs',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 224x224
        #batch_size=128,
        class_mode = 'binary'
        )

validation_generator = validation_datagen.flow_from_directory(
        './validation-cats-or-dogs/',  # This is the source directory for validation images
        target_size=(224, 224),  # All images will be resized to 224x224
        #batch_size=32,
        class_mode = 'binary'
        )

## Deep NN


In [ ]:
deep_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

deep_model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

deep_history = deep_model.fit(
      train_generator,
      steps_per_epoch=16,
      epochs=10,
      validation_data = validation_generator,
      validation_steps=8,
      verbose=1)

## Conv NN

In [ ]:
conv_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

conv_model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

conv_history = conv_model.fit(
      train_generator,
      steps_per_epoch=16,
      epochs=8,
      validation_data = validation_generator,
      validation_steps=8,
      verbose=1)

## VGG19

In [ ]:
vgg19_model = tf.keras.applications.VGG19(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None,
    classes=2,
    classifier_activation='sigmoid'
)

for layer in vgg19_model.layers:
  layer.trainable = False

top_model = tf.keras.models.Sequential([
    vgg19_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

top_model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

vg_history = top_model.fit(
      train_generator,
      steps_per_epoch=16,
      epochs=7,
      validation_data = validation_generator,
      validation_steps=8,
      verbose=1)

## ResNet

In [ ]:
resnet_model = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None,
    classes=2,
    classifier_activation='sigmoid'
)

for layer in resnet_model.layers:
  layer.trainable = False

res_top_model = tf.keras.models.Sequential([
    resnet_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

res_top_model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

res_history = res_top_model.fit(
      train_generator,
      steps_per_epoch=16,
      epochs=7,
      validation_data = validation_generator,
      validation_steps=8,
      verbose=1)

In [ ]:
def plotilka_loss_epochs(history):
  plt.plot(history['loss'], label='Test')
  plt.plot(history['val_loss'], label='Train')

  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

def plotilka_acc_epochs(history):
  plt.plot(history['accuracy'], label='Test')
  plt.plot(history['val_accuracy'], label='Train')

  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

plotilka_acc_epochs(deep_history.history)
plotilka_loss_epochs(deep_history.history)
plotilka_acc_epochs(conv_history.history)
plotilka_loss_epochs(conv_history.history)
plotilka_acc_epochs(vg_history.history)
plotilka_loss_epochs(vg_history.history)
plotilka_acc_epochs(res_history.history)
plotilka_loss_epochs(res_history.history)

#Conclusion

In conclusion, the task involved building and comparing different models for classifying images of dogs and cats from a dataset. The models implemented were:

a) A fully connected network with three hidden layers, considering one-dimensional vectors as input.

b) A convolutional neural network with two convolutional and sub-sampling units.

c) Transfer learning with the VGG19 and ResNet models, utilizing pre-trained weights, freezing the fully connected layers, and retraining on new data.

The performance of each model was compared, and it was observed that VGG19 outperformed the other models.

To address the potential issue of overfitting, the number of training epochs for models (a) and (b) was increased, and learning curves were plotted. These curves demonstrated the phenomenon of overfitting, where the models performed well on training data but struggled to generalize to new, unseen data.

In summary, VGG19 proved to be the most effective model for the given image classification task. The comparison of different architectures and the exploration of transfer learning techniques provided insights into the strengths and limitations of each approach, contributing to a comprehensive understanding of image classification models.